## The problematic eigenvalues for BDF3

Define
$$
\mu(x,\Delta x)=\left\{\left|\frac {\left|\xi(\Delta x \lambda)\right|-1}{\Delta x}\right|:\lambda\in \sigma\left(A+xB\right)\right\}.
$$

The following plots show for each value of $x\in 13,12,\cdots,-9,-10$, how the (normalized) number of problematic eigenvalues (outside of the absolute stability region of BDF3) change with respect to $\Delta x$, where the number of Fourier modes is fixed at $8000$ and $\beta=2$ for all plots. Besides, for each value of $\Delta x$, for those problematic eigenvalues, we also compute the maximum of $\mu(x,\Delta x)$.

In [ ]:
using TracyWidomBeta,LinearAlgebra,Statistics,SparseArrays,Distributions,Trapz,ApproxFun,SpecialFunctions
using RandomMatrices,Plots,LaTeXStrings,TypedTables,StatsBase,FFTW

In [ ]:
ρ = (α,z) -> (z.^(length(α)-1:-1:0))'*α
σ = (β,z) -> (z.^(length(β)-1:-1:0))'*β
R = (α,β,z) -> ρ(α,z)/σ(β,z)
function find_roots(c) # supposing that the leading order coefficient is 1
    # c contains the remaining coefficients
    r = length(c)
    A = zeros(Complex{Float64},r,r)
    A[1,:] = -c
    A[2:end,1:end-1] = A[2:end,1:end-1] + I # add identity matrix to lower-left block
    return eigvals(A)
end
function compute_roots(α,β,z)
    r = length(α)-1
    c = α-z*β
    if α[1]-z*β[1] ≈ 0.
        λ = find_roots(c[3:end]/c[2])
    else
        λ = find_roots(c[2:end]/c[1]) # let's suppose that first and second coefficients don't vanish simultaneously
    end
    return λ
end
function check_condition(λ)
    if maximum(abs.(λ)) > 1
        return 0
    else
        for i = 1:length(λ)
            if abs(λ[i]) ≈ 1. && sum(map(t -> λ[i] ≈ t,λ)) > 1
                return 0
            end
        end
    end
    return 1
end
function check_condition2(λ)
    ξ_rest=zeros(ComplexF64,0)
    for i=1:length(λ)
        if abs(λ[i])>1
            append!(ξ_rest,λ[i])
        elseif abs(λ[i]) ≈ 1. && sum(map(t -> λ[i] ≈ t,λ)) > 1
            append!(ξ_rest,λ[i])
        end
    end
    return ξ_rest
end
function root_condition(α,β,z)
    return compute_roots(α,β,z) |> check_condition
end

Construct the matrices.

In [ ]:
β=2;
M_f=1000;#just some random value
mgrid=(n,L) -> L*(1:n)/n;
θ=mgrid(M_f,pi);
M_s=8000;
h=(1/M_f)*pi;
method="spectral";
l=10;
(A,B)=TracyWidomBeta.matrix_gen(β;method,M_f,M_s,h,θ,l);

For $x=-10$, we have

In [ ]:
Delta_x=[-1,-0.5,-0.2,-0.1,-0.05,-0.04,-0.02,-0.01,-0.005,-0.004,-0.002,-0.001,-0.0005,-0.0002,-0.0001];
r_leftm10=zeros(1,length(Delta_x));r_rightm10=zeros(1,length(Delta_x));
v_leftm10=zeros(1,length(Delta_x));v_rightm10=zeros(1,length(Delta_x));
Z_left=zeros(ComplexF64,0);Z_right=zeros(ComplexF64,0);
α = [11/6,-3,3/2,-1/3];β = [1,0,0,0];
C=A-10*B;
Eig=eigvals(Matrix(C));
for j=1:length(Delta_x)
    z_left=zeros(ComplexF64,0);z_right=zeros(ComplexF64,0);
    ξ_left=zeros(ComplexF64,0);ξ_right=zeros(ComplexF64,0);
    Δx=Delta_x[j]
    z=Δx*Eig;
    for i=1:length(z)
        ξ=compute_roots(α,β,z[i])
        if real(z[i])>0 && check_condition(ξ)==0
            append!(z_right,z[i])
            append!(ξ_right,ξ)
        end
        if real(z[i])<0 && check_condition(ξ)==0
            append!(z_left,z[i])
            append!(ξ_left,ξ)
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_left,z[i])
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_left,z[i])
        end
    end
    r_leftm10[j]=length(z_left)/M_s;
    r_rightm10[j]=length(z_right)/M_s;
    if length(ξ_left)==0
        v_leftm10[j]=0
    end
    if length(ξ_right)==0
        v_rightm10[j]=0
    end
    if length(ξ_left)>0
        ξ_restl=check_condition2(ξ_left)
        μ_left=abs.((abs.(ξ_restl).-1)./Δx)
        v_leftm10[j]=maximum(μ_left)
    end
    if length(ξ_right)>0
        ξ_restr=check_condition2(ξ_right)
        μ_right=abs.((abs.(ξ_restr).-1)./Δx)
        v_rightm10[j]=maximum(μ_right)
    end
end

In [ ]:
p_m10r=Plots.plot();
p_m10r=plot!(Delta_x.*(-1),vec(r_leftm10),lw=4,label="Left-half plane",legend=:topleft,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m10r=plot!(Delta_x.*(-1),vec(r_rightm10),lw=4,label="Right-half plane",legend=:topleft,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m10v=Plots.plot();
p_m10v=plot!(Delta_x.*(-1),vec(v_leftm10),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m10v=plot!(Delta_x.*(-1),vec(v_rightm10),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
plot(p_m10r,p_m10v,size=(900,400))

In [ ]:
t_m10=Table(Δx=Delta_x,ratio_left=vec(r_leftm10),mean_left=vec(v_leftm10),ratio_right=vec(r_rightm10),mean_right=vec(v_rightm10))

For $x=-9$, we have

In [ ]:
Delta_x=[-1,-0.5,-0.2,-0.1,-0.05,-0.04,-0.02,-0.01,-0.005,-0.004,-0.002,-0.001,-0.0005,-0.0002,-0.0001];
r_leftm9=zeros(1,length(Delta_x));r_rightm9=zeros(1,length(Delta_x));
v_leftm9=zeros(1,length(Delta_x));v_rightm9=zeros(1,length(Delta_x));
α = [11/6,-3,3/2,-1/3];β = [1,0,0,0];
C=A-9*B;
Eig=eigvals(Matrix(C));
for j=1:length(Delta_x)
    z_left=zeros(ComplexF64,0);z_right=zeros(ComplexF64,0);
    ξ_left=zeros(ComplexF64,0);ξ_right=zeros(ComplexF64,0);
    Δx=Delta_x[j]
    z=Δx*Eig;
    for i=1:length(z)
        ξ=compute_roots(α,β,z[i])
        if real(z[i])>0 && check_condition(ξ)==0
            append!(z_right,z[i])
            append!(ξ_right,ξ)
        end
        if real(z[i])<0 && check_condition(ξ)==0
            append!(z_left,z[i])
            append!(ξ_left,ξ)
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_left,z[i])
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_left,z[i])
        end
    end
    r_leftm9[j]=length(z_left)/M_s;
    r_rightm9[j]=length(z_right)/M_s;
    if length(ξ_left)==0
        v_leftm9[j]=0
    end
    if length(ξ_right)==0
        v_rightm9[j]=0
    end
    if length(ξ_left)>0
        ξ_restl=check_condition2(ξ_left)
        μ_left=abs.((abs.(ξ_restl).-1)./Δx)
        v_leftm9[j]=maximum(μ_left)
    end
    if length(ξ_right)>0
        ξ_restr=check_condition2(ξ_right)
        μ_right=abs.((abs.(ξ_restr).-1)./Δx)
        v_rightm9[j]=maximum(μ_right)
    end
end

In [ ]:
p_m9r=Plots.plot();
p_m9r=plot!(Delta_x.*(-1),vec(r_leftm9),lw=4,label="Left-half plane",legend=:topleft,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m9r=plot!(Delta_x.*(-1),vec(r_rightm9),lw=4,label="Right-half plane",legend=:topleft,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m9v=Plots.plot();
p_m9v=plot!(Delta_x.*(-1),vec(v_leftm9),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m9v=plot!(Delta_x.*(-1),vec(v_rightm9),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
plot(p_m9r,p_m9v,size=(900,400))

In [ ]:
t_m9=Table(Δx=Delta_x,ratio_left=vec(r_leftm9),mean_left=vec(v_leftm9),ratio_right=vec(r_rightm9),mean_right=vec(v_rightm9))

For $x=-8$, we have

In [ ]:
Delta_x=[-1,-0.5,-0.2,-0.1,-0.05,-0.04,-0.02,-0.01,-0.005,-0.004,-0.002,-0.001,-0.0005,-0.0002,-0.0001];
r_leftm8=zeros(1,length(Delta_x));r_rightm8=zeros(1,length(Delta_x));
v_leftm8=zeros(1,length(Delta_x));v_rightm8=zeros(1,length(Delta_x));
α = [11/6,-3,3/2,-1/3];β = [1,0,0,0];
C=A-8*B;
Eig=eigvals(Matrix(C));
for j=1:length(Delta_x)
    z_left=zeros(ComplexF64,0);z_right=zeros(ComplexF64,0);
    ξ_left=zeros(ComplexF64,0);ξ_right=zeros(ComplexF64,0);
    Δx=Delta_x[j]
    z=Δx*Eig;
    for i=1:length(z)
        ξ=compute_roots(α,β,z[i])
        if real(z[i])>0 && check_condition(ξ)==0
            append!(z_right,z[i])
            append!(ξ_right,ξ)
        end
        if real(z[i])<0 && check_condition(ξ)==0
            append!(z_left,z[i])
            append!(ξ_left,ξ)
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_left,z[i])
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_left,z[i])
        end
    end
    r_leftm8[j]=length(z_left)/M_s;
    r_rightm8[j]=length(z_right)/M_s;
    if length(ξ_left)==0
        v_leftm8[j]=0
    end
    if length(ξ_right)==0
        v_rightm8[j]=0
    end
    if length(ξ_left)>0
        ξ_restl=check_condition2(ξ_left)
        μ_left=abs.((abs.(ξ_restl).-1)./Δx)
        v_leftm8[j]=maximum(μ_left)
    end
    if length(ξ_right)>0
        ξ_restr=check_condition2(ξ_right)
        μ_right=abs.((abs.(ξ_restr).-1)./Δx)
        v_rightm8[j]=maximum(μ_right)
    end
end

In [ ]:
p_m8r=Plots.plot();
p_m8r=plot!(Delta_x.*(-1),vec(r_leftm8),lw=4,label="Left-half plane",legend=:topleft,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m8r=plot!(Delta_x.*(-1),vec(r_rightm8),lw=4,label="Right-half plane",legend=:topleft,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m8v=Plots.plot();
p_m8v=plot!(Delta_x.*(-1),vec(v_leftm8),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m8v=plot!(Delta_x.*(-1),vec(v_rightm8),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
plot(p_m8r,p_m8v,size=(900,400))

In [ ]:
t_m8=Table(Δx=Delta_x,ratio_left=vec(r_leftm8),mean_left=vec(v_leftm8),ratio_right=vec(r_rightm8),mean_right=vec(v_rightm8))

For $x=-7$, we have

In [ ]:
Delta_x=[-1,-0.5,-0.2,-0.1,-0.05,-0.04,-0.02,-0.01,-0.005,-0.004,-0.002,-0.001,-0.0005,-0.0002,-0.0001];
r_leftm7=zeros(1,length(Delta_x));r_rightm7=zeros(1,length(Delta_x));
v_leftm7=zeros(1,length(Delta_x));v_rightm7=zeros(1,length(Delta_x));
α = [11/6,-3,3/2,-1/3];β = [1,0,0,0];
C=A-7*B;
Eig=eigvals(Matrix(C));
for j=1:length(Delta_x)
    z_left=zeros(ComplexF64,0);z_right=zeros(ComplexF64,0);
    ξ_left=zeros(ComplexF64,0);ξ_right=zeros(ComplexF64,0);
    Δx=Delta_x[j]
    z=Δx*Eig;
    for i=1:length(z)
        ξ=compute_roots(α,β,z[i])
        if real(z[i])>0 && check_condition(ξ)==0
            append!(z_right,z[i])
            append!(ξ_right,ξ)
        end
        if real(z[i])<0 && check_condition(ξ)==0
            append!(z_left,z[i])
            append!(ξ_left,ξ)
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_left,z[i])
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_left,z[i])
        end
    end
    r_leftm7[j]=length(z_left)/M_s;
    r_rightm7[j]=length(z_right)/M_s;
    if length(ξ_left)==0
        v_leftm7[j]=0
    end
    if length(ξ_right)==0
        v_rightm7[j]=0
    end
    if length(ξ_left)>0
        ξ_restl=check_condition2(ξ_left)
        μ_left=abs.((abs.(ξ_restl).-1)./Δx)
        v_leftm7[j]=maximum(μ_left)
    end
    if length(ξ_right)>0
        ξ_restr=check_condition2(ξ_right)
        μ_right=abs.((abs.(ξ_restr).-1)./Δx)
        v_rightm7[j]=maximum(μ_right)
    end
end

In [ ]:
p_m7r=Plots.plot();
p_m7r=plot!(Delta_x.*(-1),vec(r_leftm7),lw=4,label="Left-half plane",legend=:topleft,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m7r=plot!(Delta_x.*(-1),vec(r_rightm7),lw=4,label="Right-half plane",legend=:topleft,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m7v=Plots.plot();
p_m7v=plot!(Delta_x.*(-1),vec(v_leftm7),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m7v=plot!(Delta_x.*(-1),vec(v_rightm7),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
plot(p_m7r,p_m7v,size=(900,400))

In [ ]:
t_m7=Table(Δx=Delta_x,ratio_left=vec(r_leftm7),mean_left=vec(v_leftm7),ratio_right=vec(r_rightm7),mean_right=vec(v_rightm7))

For $x=-6$, we have

In [ ]:
Delta_x=[-1,-0.5,-0.2,-0.1,-0.05,-0.04,-0.02,-0.01,-0.005,-0.004,-0.002,-0.001,-0.0005,-0.0002,-0.0001];
r_leftm6=zeros(1,length(Delta_x));r_rightm6=zeros(1,length(Delta_x));
v_leftm6=zeros(1,length(Delta_x));v_rightm6=zeros(1,length(Delta_x));
α = [11/6,-3,3/2,-1/3];β = [1,0,0,0];
C=A-6*B;
Eig=eigvals(Matrix(C));
for j=1:length(Delta_x)
    z_left=zeros(ComplexF64,0);z_right=zeros(ComplexF64,0);
    ξ_left=zeros(ComplexF64,0);ξ_right=zeros(ComplexF64,0);
    Δx=Delta_x[j]
    z=Δx*Eig;
    for i=1:length(z)
        ξ=compute_roots(α,β,z[i])
        if real(z[i])>0 && check_condition(ξ)==0
            append!(z_right,z[i])
            append!(ξ_right,ξ)
        end
        if real(z[i])<0 && check_condition(ξ)==0
            append!(z_left,z[i])
            append!(ξ_left,ξ)
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_left,z[i])
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_left,z[i])
        end
    end
    r_leftm6[j]=length(z_left)/M_s;
    r_rightm6[j]=length(z_right)/M_s;
    if length(ξ_left)==0
        v_leftm6[j]=0
    end
    if length(ξ_right)==0
        v_rightm6[j]=0
    end
    if length(ξ_left)>0
        ξ_restl=check_condition2(ξ_left)
        μ_left=abs.((abs.(ξ_restl).-1)./Δx)
        v_leftm6[j]=maximum(μ_left)
    end
    if length(ξ_right)>0
        ξ_restr=check_condition2(ξ_right)
        μ_right=abs.((abs.(ξ_restr).-1)./Δx)
        v_rightm6[j]=maximum(μ_right)
    end
end

In [ ]:
p_m6r=Plots.plot();
p_m6r=plot!(Delta_x.*(-1),vec(r_leftm6),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m6r=plot!(Delta_x.*(-1),vec(r_rightm6),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m6v=Plots.plot();
p_m6v=plot!(Delta_x.*(-1),vec(v_leftm6),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m6v=plot!(Delta_x.*(-1),vec(v_rightm6),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
plot(p_m6r,p_m6v,size=(900,400))

In [ ]:
t_m6=Table(Δx=Delta_x,ratio_left=vec(r_leftm6),mean_left=vec(v_leftm6),ratio_right=vec(r_rightm6),mean_right=vec(v_rightm6))

For $x=-5$, we have

In [ ]:
Delta_x=[-1,-0.5,-0.2,-0.1,-0.05,-0.04,-0.02,-0.01,-0.005,-0.004,-0.002,-0.001,-0.0005,-0.0002,-0.0001];
r_leftm5=zeros(1,length(Delta_x));r_rightm5=zeros(1,length(Delta_x));
v_leftm5=zeros(1,length(Delta_x));v_rightm5=zeros(1,length(Delta_x));
α = [11/6,-3,3/2,-1/3];β = [1,0,0,0];
C=A-5*B;
Eig=eigvals(Matrix(C));
for j=1:length(Delta_x)
    z_left=zeros(ComplexF64,0);z_right=zeros(ComplexF64,0);
    ξ_left=zeros(ComplexF64,0);ξ_right=zeros(ComplexF64,0);
    Δx=Delta_x[j]
    z=Δx*Eig;
    for i=1:length(z)
        ξ=compute_roots(α,β,z[i])
        if real(z[i])>0 && check_condition(ξ)==0
            append!(z_right,z[i])
            append!(ξ_right,ξ)
        end
        if real(z[i])<0 && check_condition(ξ)==0
            append!(z_left,z[i])
            append!(ξ_left,ξ)
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_left,z[i])
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_left,z[i])
        end
    end
    r_leftm5[j]=length(z_left)/M_s;
    r_rightm5[j]=length(z_right)/M_s;
    if length(ξ_left)==0
        v_leftm5[j]=0
    end
    if length(ξ_right)==0
        v_rightm5[j]=0
    end
    if length(ξ_left)>0
        ξ_restl=check_condition2(ξ_left)
        μ_left=abs.((abs.(ξ_restl).-1)./Δx)
        v_leftm5[j]=maximum(μ_left)
    end
    if length(ξ_right)>0
        ξ_restr=check_condition2(ξ_right)
        μ_right=abs.((abs.(ξ_restr).-1)./Δx)
        v_rightm5[j]=maximum(μ_right)
    end
end

In [ ]:
p_m5r=Plots.plot();
p_m5r=plot!(Delta_x.*(-1),vec(r_leftm5),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m5r=plot!(Delta_x.*(-1),vec(r_rightm5),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m5v=Plots.plot();
p_m5v=plot!(Delta_x.*(-1),vec(v_leftm5),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m5v=plot!(Delta_x.*(-1),vec(v_rightm5),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
plot(p_m5r,p_m5v,size=(900,400))

In [ ]:
t_m5=Table(Δx=Delta_x,ratio_left=vec(r_leftm5),mean_left=vec(v_leftm5),ratio_right=vec(r_rightm5),mean_right=vec(v_rightm5))

For $x=-4$, we have

In [ ]:
Delta_x=[-1,-0.5,-0.2,-0.1,-0.05,-0.04,-0.02,-0.01,-0.005,-0.004,-0.002,-0.001,-0.0005,-0.0002,-0.0001];
r_leftm4=zeros(1,length(Delta_x));r_rightm4=zeros(1,length(Delta_x));
v_leftm4=zeros(1,length(Delta_x));v_rightm4=zeros(1,length(Delta_x));
α = [11/6,-3,3/2,-1/3];β = [1,0,0,0];
C=A-4*B;
Eig=eigvals(Matrix(C));
for j=1:length(Delta_x)
    z_left=zeros(ComplexF64,0);z_right=zeros(ComplexF64,0);
    ξ_left=zeros(ComplexF64,0);ξ_right=zeros(ComplexF64,0);
    Δx=Delta_x[j]
    z=Δx*Eig;
    for i=1:length(z)
        ξ=compute_roots(α,β,z[i])
        if real(z[i])>0 && check_condition(ξ)==0
            append!(z_right,z[i])
            append!(ξ_right,ξ)
        end
        if real(z[i])<0 && check_condition(ξ)==0
            append!(z_left,z[i])
            append!(ξ_left,ξ)
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_left,z[i])
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_left,z[i])
        end
    end
    r_leftm4[j]=length(z_left)/M_s;
    r_rightm4[j]=length(z_right)/M_s;
    if length(ξ_left)==0
        v_leftm4[j]=0
    end
    if length(ξ_right)==0
        v_rightm4[j]=0
    end
    if length(ξ_left)>0
        ξ_restl=check_condition2(ξ_left)
        μ_left=abs.((abs.(ξ_restl).-1)./Δx)
        v_leftm4[j]=maximum(μ_left)
    end
    if length(ξ_right)>0
        ξ_restr=check_condition2(ξ_right)
        μ_right=abs.((abs.(ξ_restr).-1)./Δx)
        v_rightm4[j]=maximum(μ_right)
    end
end

In [ ]:
p_m4r=Plots.plot();
p_m4r=plot!(Delta_x.*(-1),vec(r_leftm4),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m4r=plot!(Delta_x.*(-1),vec(r_rightm4),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m4v=Plots.plot();
p_m4v=plot!(Delta_x.*(-1),vec(v_leftm4),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m4v=plot!(Delta_x.*(-1),vec(v_rightm4),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
plot(p_m4r,p_m4v,size=(900,400))

In [ ]:
t_m4=Table(Δx=Delta_x,ratio_left=vec(r_leftm4),mean_left=vec(v_leftm4),ratio_right=vec(r_rightm4),mean_right=vec(v_rightm4))

For $x=-3$, we have

In [ ]:
Delta_x=[-1,-0.5,-0.2,-0.1,-0.05,-0.04,-0.02,-0.01,-0.005,-0.004,-0.002,-0.001,-0.0005,-0.0002,-0.0001];
r_leftm3=zeros(1,length(Delta_x));r_rightm3=zeros(1,length(Delta_x));
v_leftm3=zeros(1,length(Delta_x));v_rightm3=zeros(1,length(Delta_x));
α = [11/6,-3,3/2,-1/3];β = [1,0,0,0];
C=A-3*B;
Eig=eigvals(Matrix(C));
for j=1:length(Delta_x)
    z_left=zeros(ComplexF64,0);z_right=zeros(ComplexF64,0);
    ξ_left=zeros(ComplexF64,0);ξ_right=zeros(ComplexF64,0);
    Δx=Delta_x[j]
    z=Δx*Eig;
    for i=1:length(z)
        ξ=compute_roots(α,β,z[i])
        if real(z[i])>0 && check_condition(ξ)==0
            append!(z_right,z[i])
            append!(ξ_right,ξ)
        end
        if real(z[i])<0 && check_condition(ξ)==0
            append!(z_left,z[i])
            append!(ξ_left,ξ)
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_left,z[i])
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_left,z[i])
        end
    end
    r_leftm3[j]=length(z_left)/M_s;
    r_rightm3[j]=length(z_right)/M_s;
    if length(ξ_left)==0
        v_leftm3[j]=0
    end
    if length(ξ_right)==0
        v_rightm3[j]=0
    end
    if length(ξ_left)>0
        ξ_restl=check_condition2(ξ_left)
        μ_left=abs.((abs.(ξ_restl).-1)./Δx)
        v_leftm3[j]=maximum(μ_left)
    end
    if length(ξ_right)>0
        ξ_restr=check_condition2(ξ_right)
        μ_right=abs.((abs.(ξ_restr).-1)./Δx)
        v_rightm3[j]=maximum(μ_right)
    end
end

In [ ]:
p_m3r=Plots.plot();
p_m3r=plot!(Delta_x.*(-1),vec(r_leftm3),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m3r=plot!(Delta_x.*(-1),vec(r_rightm3),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m3v=Plots.plot();
p_m3v=plot!(Delta_x.*(-1),vec(v_leftm3),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m3v=plot!(Delta_x.*(-1),vec(v_rightm3),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
plot(p_m3r,p_m3v,size=(900,400))

In [ ]:
t_m3=Table(Δx=Delta_x,ratio_left=vec(r_leftm3),mean_left=vec(v_leftm3),ratio_right=vec(r_rightm3),mean_right=vec(v_rightm3))

For $x=-2$, we have

In [ ]:
Delta_x=[-1,-0.5,-0.2,-0.1,-0.05,-0.04,-0.02,-0.01,-0.005,-0.004,-0.002,-0.001,-0.0005,-0.0002,-0.0001];
r_leftm2=zeros(1,length(Delta_x));r_rightm2=zeros(1,length(Delta_x));
v_leftm2=zeros(1,length(Delta_x));v_rightm2=zeros(1,length(Delta_x));
α = [11/6,-3,3/2,-1/3];β = [1,0,0,0];
C=A-2*B;
Eig=eigvals(Matrix(C));
for j=1:length(Delta_x)
    z_left=zeros(ComplexF64,0);z_right=zeros(ComplexF64,0);
    ξ_left=zeros(ComplexF64,0);ξ_right=zeros(ComplexF64,0);
    Δx=Delta_x[j]
    z=Δx*Eig;
    for i=1:length(z)
        ξ=compute_roots(α,β,z[i])
        if real(z[i])>0 && check_condition(ξ)==0
            append!(z_right,z[i])
            append!(ξ_right,ξ)
        end
        if real(z[i])<0 && check_condition(ξ)==0
            append!(z_left,z[i])
            append!(ξ_left,ξ)
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_left,z[i])
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_left,z[i])
        end
    end
    r_leftm2[j]=length(z_left)/M_s;
    r_rightm2[j]=length(z_right)/M_s;
    if length(ξ_left)==0
        v_leftm2[j]=0
    end
    if length(ξ_right)==0
        v_rightm2[j]=0
    end
    if length(ξ_left)>0
        ξ_restl=check_condition2(ξ_left)
        μ_left=abs.((abs.(ξ_restl).-1)./Δx)
        v_leftm2[j]=maximum(μ_left)
    end
    if length(ξ_right)>0
        ξ_restr=check_condition2(ξ_right)
        μ_right=abs.((abs.(ξ_restr).-1)./Δx)
        v_rightm2[j]=maximum(μ_right)
    end
end

In [ ]:
p_m2r=Plots.plot();
p_m2r=plot!(Delta_x.*(-1),vec(r_leftm2),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m2r=plot!(Delta_x.*(-1),vec(r_rightm2),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m2v=Plots.plot();
p_m2v=plot!(Delta_x.*(-1),vec(v_leftm2),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m2v=plot!(Delta_x.*(-1),vec(v_rightm2),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
plot(p_m2r,p_m2v,size=(900,400))

In [ ]:
t_m2=Table(Δx=Delta_x,ratio_left=vec(r_leftm2),mean_left=vec(v_leftm2),ratio_right=vec(r_rightm2),mean_right=vec(v_rightm2))

For $x=-1$, we have

In [ ]:
Delta_x=[-1,-0.5,-0.2,-0.1,-0.05,-0.04,-0.02,-0.01,-0.005,-0.004,-0.002,-0.001,-0.0005,-0.0002,-0.0001];
r_leftm1=zeros(1,length(Delta_x));r_rightm1=zeros(1,length(Delta_x));
v_leftm1=zeros(1,length(Delta_x));v_rightm1=zeros(1,length(Delta_x));
α = [11/6,-3,3/2,-1/3];β = [1,0,0,0];
C=A-1*B;
Eig=eigvals(Matrix(C));
for j=1:length(Delta_x)
    z_left=zeros(ComplexF64,0);z_right=zeros(ComplexF64,0);
    ξ_left=zeros(ComplexF64,0);ξ_right=zeros(ComplexF64,0);
    Δx=Delta_x[j]
    z=Δx*Eig;
    for i=1:length(z)
        ξ=compute_roots(α,β,z[i])
        if real(z[i])>0 && check_condition(ξ)==0
            append!(z_right,z[i])
            append!(ξ_right,ξ)
        end
        if real(z[i])<0 && check_condition(ξ)==0
            append!(z_left,z[i])
            append!(ξ_left,ξ)
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_left,z[i])
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_left,z[i])
        end
    end
    r_leftm1[j]=length(z_left)/M_s;
    r_rightm1[j]=length(z_right)/M_s;
    if length(ξ_left)==0
        v_leftm1[j]=0
    end
    if length(ξ_right)==0
        v_rightm1[j]=0
    end
    if length(ξ_left)>0
        ξ_restl=check_condition2(ξ_left)
        μ_left=abs.((abs.(ξ_restl).-1)./Δx)
        v_leftm1[j]=maximum(μ_left)
    end
    if length(ξ_right)>0
        ξ_restr=check_condition2(ξ_right)
        μ_right=abs.((abs.(ξ_restr).-1)./Δx)
        v_rightm1[j]=maximum(μ_right)
    end
end

In [ ]:
p_m1r=Plots.plot();
p_m1r=plot!(Delta_x.*(-1),vec(r_leftm1),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m1r=plot!(Delta_x.*(-1),vec(r_rightm1),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m1v=Plots.plot();
p_m1v=plot!(Delta_x.*(-1),vec(v_leftm1),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_m1v=plot!(Delta_x.*(-1),vec(v_rightm1),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
plot(p_m1r,p_m1v,size=(900,400))

In [ ]:
t_m1=Table(Δx=Delta_x,ratio_left=vec(r_leftm1),mean_left=vec(v_leftm1),ratio_right=vec(r_rightm1),mean_right=vec(v_rightm1))

For $x=0$, we have

In [ ]:
Delta_x=[-1,-0.5,-0.2,-0.1,-0.05,-0.04,-0.02,-0.01,-0.005,-0.004,-0.002,-0.001,-0.0005,-0.0002,-0.0001];
r_left0=zeros(1,length(Delta_x));r_right0=zeros(1,length(Delta_x));
v_left0=zeros(1,length(Delta_x));v_right0=zeros(1,length(Delta_x));
α = [11/6,-3,3/2,-1/3];β = [1,0,0,0];
C=A-0*B;
Eig=eigvals(Matrix(C));
for j=1:length(Delta_x)
    z_left=zeros(ComplexF64,0);z_right=zeros(ComplexF64,0);
    ξ_left=zeros(ComplexF64,0);ξ_right=zeros(ComplexF64,0);
    Δx=Delta_x[j]
    z=Δx*Eig;
    for i=1:length(z)
        ξ=compute_roots(α,β,z[i])
        if real(z[i])>0 && check_condition(ξ)==0
            append!(z_right,z[i])
            append!(ξ_right,ξ)
        end
        if real(z[i])<0 && check_condition(ξ)==0
            append!(z_left,z[i])
            append!(ξ_left,ξ)
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_left,z[i])
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_left,z[i])
        end
    end
    r_left0[j]=length(z_left)/M_s;
    r_right0[j]=length(z_right)/M_s;
    if length(ξ_left)==0
        v_left0[j]=0
    end
    if length(ξ_right)==0
        v_right0[j]=0
    end
    if length(ξ_left)>0
        ξ_restl=check_condition2(ξ_left)
        μ_left=abs.((abs.(ξ_restl).-1)./Δx)
        v_left0[j]=maximum(μ_left)
    end
    if length(ξ_right)>0
        ξ_restr=check_condition2(ξ_right)
        μ_right=abs.((abs.(ξ_restr).-1)./Δx)
        v_right0[j]=maximum(μ_right)
    end
end

In [ ]:
p_0r=Plots.plot();
p_0r=plot!(Delta_x.*(-1),vec(r_left0),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_0r=plot!(Delta_x.*(-1),vec(r_right0),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_0v=Plots.plot();
p_0v=plot!(Delta_x.*(-1),vec(v_left0),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_0v=plot!(Delta_x.*(-1),vec(v_right0),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
plot(p_0r,p_0v,size=(900,400))

In [ ]:
t_0=Table(Δx=Delta_x,ratio_left=vec(r_left0),mean_left=vec(v_left0),ratio_right=vec(r_right0),mean_right=vec(v_right0))

For $x=1$, we have

In [ ]:
Delta_x=[-1,-0.5,-0.2,-0.1,-0.05,-0.04,-0.02,-0.01,-0.005,-0.004,-0.002,-0.001,-0.0005,-0.0002,-0.0001];
r_left1=zeros(1,length(Delta_x));r_right1=zeros(1,length(Delta_x));
v_left1=zeros(1,length(Delta_x));v_right1=zeros(1,length(Delta_x));
α = [11/6,-3,3/2,-1/3];β = [1,0,0,0];
C=A+1*B;
Eig=eigvals(Matrix(C));
for j=1:length(Delta_x)
    z_left=zeros(ComplexF64,0);z_right=zeros(ComplexF64,0);
    ξ_left=zeros(ComplexF64,0);ξ_right=zeros(ComplexF64,0);
    Δx=Delta_x[j]
    z=Δx*Eig;
    for i=1:length(z)
        ξ=compute_roots(α,β,z[i])
        if real(z[i])>0 && check_condition(ξ)==0
            append!(z_right,z[i])
            append!(ξ_right,ξ)
        end
        if real(z[i])<0 && check_condition(ξ)==0
            append!(z_left,z[i])
            append!(ξ_left,ξ)
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_left,z[i])
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_left,z[i])
        end
    end
    r_left1[j]=length(z_left)/M_s;
    r_right1[j]=length(z_right)/M_s;
    if length(ξ_left)==0
        v_left1[j]=0
    end
    if length(ξ_right)==0
        v_right1[j]=0
    end
    if length(ξ_left)>0
        ξ_restl=check_condition2(ξ_left)
        μ_left=abs.((abs.(ξ_restl).-1)./Δx)
        v_left1[j]=maximum(μ_left)
    end
    if length(ξ_right)>0
        ξ_restr=check_condition2(ξ_right)
        μ_right=abs.((abs.(ξ_restr).-1)./Δx)
        v_right1[j]=maximum(μ_right)
    end
end

In [ ]:
p_1r=Plots.plot();
p_1r=plot!(Delta_x.*(-1),vec(r_left1),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_1r=plot!(Delta_x.*(-1),vec(r_right1),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_1v=Plots.plot();
p_1v=plot!(Delta_x.*(-1),vec(v_left1),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_1v=plot!(Delta_x.*(-1),vec(v_right1),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
plot(p_1r,p_1v,size=(900,400))

In [ ]:
t_1=Table(Δx=Delta_x,ratio_left=vec(r_left1),mean_left=vec(v_left1),ratio_right=vec(r_right1),mean_right=vec(v_right1))

For $x=2$, we have

In [ ]:
Delta_x=[-1,-0.5,-0.2,-0.1,-0.05,-0.04,-0.02,-0.01,-0.005,-0.004,-0.002,-0.001,-0.0005,-0.0002,-0.0001];
r_left2=zeros(1,length(Delta_x));r_right2=zeros(1,length(Delta_x));
v_left2=zeros(1,length(Delta_x));v_right2=zeros(1,length(Delta_x));
α = [11/6,-3,3/2,-1/3];β = [1,0,0,0];
C=A+2*B;
Eig=eigvals(Matrix(C));
for j=1:length(Delta_x)
    z_left=zeros(ComplexF64,0);z_right=zeros(ComplexF64,0);
    ξ_left=zeros(ComplexF64,0);ξ_right=zeros(ComplexF64,0);
    Δx=Delta_x[j]
    z=Δx*Eig;
    for i=1:length(z)
        ξ=compute_roots(α,β,z[i])
        if real(z[i])>0 && check_condition(ξ)==0
            append!(z_right,z[i])
            append!(ξ_right,ξ)
        end
        if real(z[i])<0 && check_condition(ξ)==0
            append!(z_left,z[i])
            append!(ξ_left,ξ)
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_left,z[i])
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_left,z[i])
        end
    end
    r_left2[j]=length(z_left)/M_s;
    r_right2[j]=length(z_right)/M_s;
    if length(ξ_left)==0
        v_left2[j]=0
    end
    if length(ξ_right)==0
        v_right2[j]=0
    end
    if length(ξ_left)>0
        ξ_restl=check_condition2(ξ_left)
        μ_left=abs.((abs.(ξ_restl).-1)./Δx)
        v_left2[j]=maximum(μ_left)
    end
    if length(ξ_right)>0
        ξ_restr=check_condition2(ξ_right)
        μ_right=abs.((abs.(ξ_restr).-1)./Δx)
        v_right2[j]=maximum(μ_right)
    end
end

In [ ]:
p_2r=Plots.plot();
p_2r=plot!(Delta_x.*(-1),vec(r_left1),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_2r=plot!(Delta_x.*(-1),vec(r_right1),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_2v=Plots.plot();
p_2v=plot!(Delta_x.*(-1),vec(v_left1),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_2v=plot!(Delta_x.*(-1),vec(v_right1),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
plot(p_2r,p_2v,size=(900,400))

In [ ]:
t_2=Table(Δx=Delta_x,ratio_left=vec(r_left2),mean_left=vec(v_left2),ratio_right=vec(r_right2),mean_right=vec(v_right2))

For $x=3$, we have

In [ ]:
Delta_x=[-1,-0.5,-0.2,-0.1,-0.05,-0.04,-0.02,-0.01,-0.005,-0.004,-0.002,-0.001,-0.0005,-0.0002,-0.0001];
r_left3=zeros(1,length(Delta_x));r_right3=zeros(1,length(Delta_x));
v_left3=zeros(1,length(Delta_x));v_right3=zeros(1,length(Delta_x));
α = [11/6,-3,3/2,-1/3];β = [1,0,0,0];
C=A+3*B;
Eig=eigvals(Matrix(C));
for j=1:length(Delta_x)
    z_left=zeros(ComplexF64,0);z_right=zeros(ComplexF64,0);
    ξ_left=zeros(ComplexF64,0);ξ_right=zeros(ComplexF64,0);
    Δx=Delta_x[j]
    z=Δx*Eig;
    for i=1:length(z)
        ξ=compute_roots(α,β,z[i])
        if real(z[i])>0 && check_condition(ξ)==0
            append!(z_right,z[i])
            append!(ξ_right,ξ)
        end
        if real(z[i])<0 && check_condition(ξ)==0
            append!(z_left,z[i])
            append!(ξ_left,ξ)
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_left,z[i])
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_left,z[i])
        end
    end
    r_left3[j]=length(z_left)/M_s;
    r_right3[j]=length(z_right)/M_s;
    if length(ξ_left)==0
        v_left3[j]=0
    end
    if length(ξ_right)==0
        v_right3[j]=0
    end
    if length(ξ_left)>0
        ξ_restl=check_condition2(ξ_left)
        μ_left=abs.((abs.(ξ_restl).-1)./Δx)
        v_left3[j]=maximum(μ_left)
    end
    if length(ξ_right)>0
        ξ_restr=check_condition2(ξ_right)
        μ_right=abs.((abs.(ξ_restr).-1)./Δx)
        v_right3[j]=maximum(μ_right)
    end
end

In [ ]:
p_3r=Plots.plot();
p_3r=plot!(Delta_x.*(-1),vec(r_left3),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_3r=plot!(Delta_x.*(-1),vec(r_right3),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_3v=Plots.plot();
p_3v=plot!(Delta_x.*(-1),vec(v_left3),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_3v=plot!(Delta_x.*(-1),vec(v_right3),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
plot(p_3r,p_3v,size=(900,400))

In [ ]:
t_3=Table(Δx=Delta_x,ratio_left=vec(r_left3),mean_left=vec(v_left3),ratio_right=vec(r_right3),mean_right=vec(v_right3))

For $x=4$, we have

In [ ]:
Delta_x=[-1,-0.5,-0.2,-0.1,-0.05,-0.04,-0.02,-0.01,-0.005,-0.004,-0.002,-0.001,-0.0005,-0.0002,-0.0001];
r_left4=zeros(1,length(Delta_x));r_right4=zeros(1,length(Delta_x));
v_left4=zeros(1,length(Delta_x));v_right4=zeros(1,length(Delta_x));
α = [11/6,-3,3/2,-1/3];β = [1,0,0,0];
C=A+4*B;
Eig=eigvals(Matrix(C));
for j=1:length(Delta_x)
    z_left=zeros(ComplexF64,0);z_right=zeros(ComplexF64,0);
    ξ_left=zeros(ComplexF64,0);ξ_right=zeros(ComplexF64,0);
    Δx=Delta_x[j]
    z=Δx*Eig;
    for i=1:length(z)
        ξ=compute_roots(α,β,z[i])
        if real(z[i])>0 && check_condition(ξ)==0
            append!(z_right,z[i])
            append!(ξ_right,ξ)
        end
        if real(z[i])<0 && check_condition(ξ)==0
            append!(z_left,z[i])
            append!(ξ_left,ξ)
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_left,z[i])
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_left,z[i])
        end
    end
    r_left4[j]=length(z_left)/M_s;
    r_right4[j]=length(z_right)/M_s;
    if length(ξ_left)==0
        v_left4[j]=0
    end
    if length(ξ_right)==0
        v_right4[j]=0
    end
    if length(ξ_left)>0
        ξ_restl=check_condition2(ξ_left)
        μ_left=abs.((abs.(ξ_restl).-1)./Δx)
        v_left4[j]=maximum(μ_left)
    end
    if length(ξ_right)>0
        ξ_restr=check_condition2(ξ_right)
        μ_right=abs.((abs.(ξ_restr).-1)./Δx)
        v_right4[j]=maximum(μ_right)
    end
end

In [ ]:
p_4r=Plots.plot();
p_4r=plot!(Delta_x.*(-1),vec(r_left4),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_4r=plot!(Delta_x.*(-1),vec(r_right4),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_4v=Plots.plot();
p_4v=plot!(Delta_x.*(-1),vec(v_left4),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_4v=plot!(Delta_x.*(-1),vec(v_right4),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
plot(p_4r,p_4v,size=(900,400))

In [ ]:
t_4=Table(Δx=Delta_x,ratio_left=vec(r_left4),mean_left=vec(v_left4),ratio_right=vec(r_right4),mean_right=vec(v_right4))

For $x=5$, we have

In [ ]:
Delta_x=[-1,-0.5,-0.2,-0.1,-0.05,-0.04,-0.02,-0.01,-0.005,-0.004,-0.002,-0.001,-0.0005,-0.0002,-0.0001];
r_left5=zeros(1,length(Delta_x));r_right5=zeros(1,length(Delta_x));
v_left5=zeros(1,length(Delta_x));v_right5=zeros(1,length(Delta_x));
α = [11/6,-3,3/2,-1/3];β = [1,0,0,0];
C=A+5*B;
Eig=eigvals(Matrix(C));
for j=1:length(Delta_x)
    z_left=zeros(ComplexF64,0);z_right=zeros(ComplexF64,0);
    ξ_left=zeros(ComplexF64,0);ξ_right=zeros(ComplexF64,0);
    Δx=Delta_x[j]
    z=Δx*Eig;
    for i=1:length(z)
        ξ=compute_roots(α,β,z[i])
        if real(z[i])>0 && check_condition(ξ)==0
            append!(z_right,z[i])
            append!(ξ_right,ξ)
        end
        if real(z[i])<0 && check_condition(ξ)==0
            append!(z_left,z[i])
            append!(ξ_left,ξ)
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_left,z[i])
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_left,z[i])
        end
    end
    r_left5[j]=length(z_left)/M_s;
    r_right5[j]=length(z_right)/M_s;
    if length(ξ_left)==0
        v_left5[j]=0
    end
    if length(ξ_right)==0
        v_right5[j]=0
    end
    if length(ξ_left)>0
        ξ_restl=check_condition2(ξ_left)
        μ_left=abs.((abs.(ξ_restl).-1)./Δx)
        v_left5[j]=maximum(μ_left)
    end
    if length(ξ_right)>0
        ξ_restr=check_condition2(ξ_right)
        μ_right=abs.((abs.(ξ_restr).-1)./Δx)
        v_right5[j]=maximum(μ_right)
    end
end

In [ ]:
p_5r=Plots.plot();
p_5r=plot!(Delta_x.*(-1),vec(r_left5),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_5r=plot!(Delta_x.*(-1),vec(r_right5),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_5v=Plots.plot();
p_5v=plot!(Delta_x.*(-1),vec(v_left5),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_5v=plot!(Delta_x.*(-1),vec(v_right5),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
plot(p_5r,p_5v,size=(900,400))

In [ ]:
t_5=Table(Δx=Delta_x,ratio_left=vec(r_left5),mean_left=vec(v_left5),ratio_right=vec(r_right5),mean_right=vec(v_right5))

For $x=6$, we have

In [ ]:
Delta_x=[-1,-0.5,-0.2,-0.1,-0.05,-0.04,-0.02,-0.01,-0.005,-0.004,-0.002,-0.001,-0.0005,-0.0002,-0.0001];
r_left6=zeros(1,length(Delta_x));r_right6=zeros(1,length(Delta_x));
v_left6=zeros(1,length(Delta_x));v_right6=zeros(1,length(Delta_x));
α = [11/6,-3,3/2,-1/3];β = [1,0,0,0];
C=A+6*B;
Eig=eigvals(Matrix(C));
for j=1:length(Delta_x)
    z_left=zeros(ComplexF64,0);z_right=zeros(ComplexF64,0);
    ξ_left=zeros(ComplexF64,0);ξ_right=zeros(ComplexF64,0);
    Δx=Delta_x[j]
    z=Δx*Eig;
    for i=1:length(z)
        ξ=compute_roots(α,β,z[i])
        if real(z[i])>0 && check_condition(ξ)==0
            append!(z_right,z[i])
            append!(ξ_right,ξ)
        end
        if real(z[i])<0 && check_condition(ξ)==0
            append!(z_left,z[i])
            append!(ξ_left,ξ)
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_left,z[i])
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_left,z[i])
        end
    end
    r_left6[j]=length(z_left)/M_s;
    r_right6[j]=length(z_right)/M_s;
    if length(ξ_left)==0
        v_left6[j]=0
    end
    if length(ξ_right)==0
        v_right6[j]=0
    end
    if length(ξ_left)>0
        ξ_restl=check_condition2(ξ_left)
        μ_left=abs.((abs.(ξ_restl).-1)./Δx)
        v_left6[j]=maximum(μ_left)
    end
    if length(ξ_right)>0
        ξ_restr=check_condition2(ξ_right)
        μ_right=abs.((abs.(ξ_restr).-1)./Δx)
        v_right6[j]=maximum(μ_right)
    end
end

In [ ]:
p_6r=Plots.plot();
p_6r=plot!(Delta_x.*(-1),vec(r_left6),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_6r=plot!(Delta_x.*(-1),vec(r_right6),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_6v=Plots.plot();
p_6v=plot!(Delta_x.*(-1),vec(v_left6),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_6v=plot!(Delta_x.*(-1),vec(v_right6),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
plot(p_6r,p_6v,size=(900,400))

In [ ]:
t_6=Table(Δx=Delta_x,ratio_left=vec(r_left6),mean_left=vec(v_left6),ratio_right=vec(r_right6),mean_right=vec(v_right6))

For $x=7$, we have

In [ ]:
Delta_x=[-1,-0.5,-0.2,-0.1,-0.05,-0.04,-0.02,-0.01,-0.005,-0.004,-0.002,-0.001,-0.0005,-0.0002,-0.0001];
r_left7=zeros(1,length(Delta_x));r_right7=zeros(1,length(Delta_x));
v_left7=zeros(1,length(Delta_x));v_right7=zeros(1,length(Delta_x));
α = [11/6,-3,3/2,-1/3];β = [1,0,0,0];
C=A+7*B;
Eig=eigvals(Matrix(C));
for j=1:length(Delta_x)
    z_left=zeros(ComplexF64,0);z_right=zeros(ComplexF64,0);
    ξ_left=zeros(ComplexF64,0);ξ_right=zeros(ComplexF64,0);
    Δx=Delta_x[j]
    z=Δx*Eig;
    for i=1:length(z)
        ξ=compute_roots(α,β,z[i])
        if real(z[i])>0 && check_condition(ξ)==0
            append!(z_right,z[i])
            append!(ξ_right,ξ)
        end
        if real(z[i])<0 && check_condition(ξ)==0
            append!(z_left,z[i])
            append!(ξ_left,ξ)
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_left,z[i])
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_left,z[i])
        end
    end
    r_left7[j]=length(z_left)/M_s;
    r_right7[j]=length(z_right)/M_s;
    if length(ξ_left)==0
        v_left7[j]=0
    end
    if length(ξ_right)==0
        v_right7[j]=0
    end
    if length(ξ_left)>0
        ξ_restl=check_condition2(ξ_left)
        μ_left=abs.((abs.(ξ_restl).-1)./Δx)
        v_left7[j]=maximum(μ_left)
    end
    if length(ξ_right)>0
        ξ_restr=check_condition2(ξ_right)
        μ_right=abs.((abs.(ξ_restr).-1)./Δx)
        v_right7[j]=maximum(μ_right)
    end
end

In [ ]:
p_7r=Plots.plot();
p_7r=plot!(Delta_x.*(-1),vec(r_left7),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_7r=plot!(Delta_x.*(-1),vec(r_right7),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_7v=Plots.plot();
p_7v=plot!(Delta_x.*(-1),vec(v_left7),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_7v=plot!(Delta_x.*(-1),vec(v_right7),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
plot(p_7r,p_7v,size=(900,400))

In [ ]:
t_7=Table(Δx=Delta_x,ratio_left=vec(r_left7),mean_left=vec(v_left7),ratio_right=vec(r_right7),mean_right=vec(v_right7))

For $x=8$, we have

In [ ]:
Delta_x=[-1,-0.5,-0.2,-0.1,-0.05,-0.04,-0.02,-0.01,-0.005,-0.004,-0.002,-0.001,-0.0005,-0.0002,-0.0001];
r_left8=zeros(1,length(Delta_x));r_right8=zeros(1,length(Delta_x));
v_left8=zeros(1,length(Delta_x));v_right8=zeros(1,length(Delta_x));
α = [11/6,-3,3/2,-1/3];β = [1,0,0,0];
C=A+8*B;
Eig=eigvals(Matrix(C));
for j=1:length(Delta_x)
    z_left=zeros(ComplexF64,0);z_right=zeros(ComplexF64,0);
    ξ_left=zeros(ComplexF64,0);ξ_right=zeros(ComplexF64,0);
    Δx=Delta_x[j]
    z=Δx*Eig;
    for i=1:length(z)
        ξ=compute_roots(α,β,z[i])
        if real(z[i])>0 && check_condition(ξ)==0
            append!(z_right,z[i])
            append!(ξ_right,ξ)
        end
        if real(z[i])<0 && check_condition(ξ)==0
            append!(z_left,z[i])
            append!(ξ_left,ξ)
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_left,z[i])
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_left,z[i])
        end
    end
    r_left8[j]=length(z_left)/M_s;
    r_right8[j]=length(z_right)/M_s;
    if length(ξ_left)==0
        v_left8[j]=0
    end
    if length(ξ_right)==0
        v_right8[j]=0
    end
    if length(ξ_left)>0
        ξ_restl=check_condition2(ξ_left)
        μ_left=abs.((abs.(ξ_restl).-1)./Δx)
        v_left8[j]=maximum(μ_left)
    end
    if length(ξ_right)>0
        ξ_restr=check_condition2(ξ_right)
        μ_right=abs.((abs.(ξ_restr).-1)./Δx)
        v_right8[j]=maximum(μ_right)
    end
end

In [ ]:
p_8r=Plots.plot();
p_8r=plot!(Delta_x.*(-1),vec(r_left8),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_8r=plot!(Delta_x.*(-1),vec(r_right8),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_8v=Plots.plot();
p_8v=plot!(Delta_x.*(-1),vec(v_left8),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_8v=plot!(Delta_x.*(-1),vec(v_right8),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
plot(p_8r,p_8v,size=(900,400))

In [ ]:
t_8=Table(Δx=Delta_x,ratio_left=vec(r_left8),mean_left=vec(v_left8),ratio_right=vec(r_right8),mean_right=vec(v_right8))

For $x=9$, we have

In [ ]:
Delta_x=[-1,-0.5,-0.2,-0.1,-0.05,-0.04,-0.02,-0.01,-0.005,-0.004,-0.002,-0.001,-0.0005,-0.0002,-0.0001];
r_left9=zeros(1,length(Delta_x));r_right9=zeros(1,length(Delta_x));
v_left9=zeros(1,length(Delta_x));v_right9=zeros(1,length(Delta_x));
α = [11/6,-3,3/2,-1/3];β = [1,0,0,0];
C=A+9*B;
Eig=eigvals(Matrix(C));
for j=1:length(Delta_x)
    z_left=zeros(ComplexF64,0);z_right=zeros(ComplexF64,0);
    ξ_left=zeros(ComplexF64,0);ξ_right=zeros(ComplexF64,0);
    Δx=Delta_x[j]
    z=Δx*Eig;
    for i=1:length(z)
        ξ=compute_roots(α,β,z[i])
        if real(z[i])>0 && check_condition(ξ)==0
            append!(z_right,z[i])
            append!(ξ_right,ξ)
        end
        if real(z[i])<0 && check_condition(ξ)==0
            append!(z_left,z[i])
            append!(ξ_left,ξ)
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_left,z[i])
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_left,z[i])
        end
    end
    r_left9[j]=length(z_left)/M_s;
    r_right9[j]=length(z_right)/M_s;
    if length(ξ_left)==0
        v_left9[j]=0
    end
    if length(ξ_right)==0
        v_right9[j]=0
    end
    if length(ξ_left)>0
        ξ_restl=check_condition2(ξ_left)
        μ_left=abs.((abs.(ξ_restl).-1)./Δx)
        v_left9[j]=maximum(μ_left)
    end
    if length(ξ_right)>0
        ξ_restr=check_condition2(ξ_right)
        μ_right=abs.((abs.(ξ_restr).-1)./Δx)
        v_right9[j]=maximum(μ_right)
    end
end

In [ ]:
p_9r=Plots.plot();
p_9r=plot!(Delta_x.*(-1),vec(r_left9),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_9r=plot!(Delta_x.*(-1),vec(r_right9),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_9v=Plots.plot();
p_9v=plot!(Delta_x.*(-1),vec(v_left9),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_9v=plot!(Delta_x.*(-1),vec(v_right9),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
plot(p_9r,p_9v,size=(900,400))

In [ ]:
t_9=Table(Δx=Delta_x,ratio_left=vec(r_left9),mean_left=vec(v_left9),ratio_right=vec(r_right9),mean_right=vec(v_right9))

For $x=10$, we have

In [ ]:
Delta_x=[-1,-0.5,-0.2,-0.1,-0.05,-0.04,-0.02,-0.01,-0.005,-0.004,-0.002,-0.001,-0.0005,-0.0002,-0.0001];
r_left10=zeros(1,length(Delta_x));r_right10=zeros(1,length(Delta_x));
v_left10=zeros(1,length(Delta_x));v_right10=zeros(1,length(Delta_x));
α = [11/6,-3,3/2,-1/3];β = [1,0,0,0];
C=A+10*B;
Eig=eigvals(Matrix(C));
for j=1:length(Delta_x)
    z_left=zeros(ComplexF64,0);z_right=zeros(ComplexF64,0);
    ξ_left=zeros(ComplexF64,0);ξ_right=zeros(ComplexF64,0);
    Δx=Delta_x[j]
    z=Δx*Eig;
    for i=1:length(z)
        ξ=compute_roots(α,β,z[i])
        if real(z[i])>0 && check_condition(ξ)==0
            append!(z_right,z[i])
            append!(ξ_right,ξ)
        end
        if real(z[i])<0 && check_condition(ξ)==0
            append!(z_left,z[i])
            append!(ξ_left,ξ)
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_left,z[i])
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_left,z[i])
        end
    end
    r_left10[j]=length(z_left)/M_s;
    r_right10[j]=length(z_right)/M_s;
    if length(ξ_left)==0
        v_left10[j]=0
    end
    if length(ξ_right)==0
        v_right10[j]=0
    end
    if length(ξ_left)>0
        ξ_restl=check_condition2(ξ_left)
        μ_left=abs.((abs.(ξ_restl).-1)./Δx)
        v_left10[j]=maximum(μ_left)
    end
    if length(ξ_right)>0
        ξ_restr=check_condition2(ξ_right)
        μ_right=abs.((abs.(ξ_restr).-1)./Δx)
        v_right10[j]=maximum(μ_right)
    end
end

In [ ]:
p_10r=Plots.plot();
p_10r=plot!(Delta_x.*(-1),vec(r_left10),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_10r=plot!(Delta_x.*(-1),vec(r_right10),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_10v=Plots.plot();
p_10v=plot!(Delta_x.*(-1),vec(v_left10),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_10v=plot!(Delta_x.*(-1),vec(v_right10),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
plot(p_10r,p_10v,size=(900,400))

In [ ]:
t_10=Table(Δx=Delta_x,ratio_left=vec(r_left10),mean_left=vec(v_left10),ratio_right=vec(r_right10),mean_right=vec(v_right10))

For $x=11$, we have

In [ ]:
Delta_x=[-1,-0.5,-0.2,-0.1,-0.05,-0.04,-0.02,-0.01,-0.005,-0.004,-0.002,-0.001,-0.0005,-0.0002,-0.0001];
r_left11=zeros(1,length(Delta_x));r_right11=zeros(1,length(Delta_x));
v_left11=zeros(1,length(Delta_x));v_right11=zeros(1,length(Delta_x));
α = [11/6,-3,3/2,-1/3];β = [1,0,0,0];
C=A+11*B;
Eig=eigvals(Matrix(C));
for j=1:length(Delta_x)
    z_left=zeros(ComplexF64,0);z_right=zeros(ComplexF64,0);
    ξ_left=zeros(ComplexF64,0);ξ_right=zeros(ComplexF64,0);
    Δx=Delta_x[j]
    z=Δx*Eig;
    for i=1:length(z)
        ξ=compute_roots(α,β,z[i])
        if real(z[i])>0 && check_condition(ξ)==0
            append!(z_right,z[i])
            append!(ξ_right,ξ)
        end
        if real(z[i])<0 && check_condition(ξ)==0
            append!(z_left,z[i])
            append!(ξ_left,ξ)
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_left,z[i])
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_left,z[i])
        end
    end
    r_left11[j]=length(z_left)/M_s;
    r_right11[j]=length(z_right)/M_s;
    if length(ξ_left)==0
        v_left11[j]=0
    end
    if length(ξ_right)==0
        v_right11[j]=0
    end
    if length(ξ_left)>0
        ξ_restl=check_condition2(ξ_left)
        μ_left=abs.((abs.(ξ_restl).-1)./Δx)
        v_left11[j]=maximum(μ_left)
    end
    if length(ξ_right)>0
        ξ_restr=check_condition2(ξ_right)
        μ_right=abs.((abs.(ξ_restr).-1)./Δx)
        v_right11[j]=maximum(μ_right)
    end
end

In [ ]:
p_11r=Plots.plot();
p_11r=plot!(Delta_x.*(-1),vec(r_left11),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_11r=plot!(Delta_x.*(-1),vec(r_right11),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_11v=Plots.plot();
p_11v=plot!(Delta_x.*(-1),vec(v_left11),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_11v=plot!(Delta_x.*(-1),vec(v_right11),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
plot(p_11r,p_11v,size=(900,400))

In [ ]:
t_11=Table(Δx=Delta_x,ratio_left=vec(r_left11),mean_left=vec(v_left11),ratio_right=vec(r_right11),mean_right=vec(v_right11))

For $x=12$, we have

In [ ]:
Delta_x=[-1,-0.5,-0.2,-0.1,-0.05,-0.04,-0.02,-0.01,-0.005,-0.004,-0.002,-0.001,-0.0005,-0.0002,-0.0001];
r_left12=zeros(1,length(Delta_x));r_right12=zeros(1,length(Delta_x));
v_left12=zeros(1,length(Delta_x));v_right12=zeros(1,length(Delta_x));
α = [11/6,-3,3/2,-1/3];β = [1,0,0,0];
C=A+12*B;
Eig=eigvals(Matrix(C));
for j=1:length(Delta_x)
    z_left=zeros(ComplexF64,0);z_right=zeros(ComplexF64,0);
    ξ_left=zeros(ComplexF64,0);ξ_right=zeros(ComplexF64,0);
    Δx=Delta_x[j]
    z=Δx*Eig;
    for i=1:length(z)
        ξ=compute_roots(α,β,z[i])
        if real(z[i])>0 && check_condition(ξ)==0
            append!(z_right,z[i])
            append!(ξ_right,ξ)
        end
        if real(z[i])<0 && check_condition(ξ)==0
            append!(z_left,z[i])
            append!(ξ_left,ξ)
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_left,z[i])
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_left,z[i])
        end
    end
    r_left12[j]=length(z_left)/M_s;
    r_right12[j]=length(z_right)/M_s;
    if length(ξ_left)==0
        v_left12[j]=0
    end
    if length(ξ_right)==0
        v_right12[j]=0
    end
    if length(ξ_left)>0
        ξ_restl=check_condition2(ξ_left)
        μ_left=abs.((abs.(ξ_restl).-1)./Δx)
        v_left12[j]=maximum(μ_left)
    end
    if length(ξ_right)>0
        ξ_restr=check_condition2(ξ_right)
        μ_right=abs.((abs.(ξ_restr).-1)./Δx)
        v_right12[j]=maximum(μ_right)
    end
end

In [ ]:
p_12r=Plots.plot();
p_12r=plot!(Delta_x.*(-1),vec(r_left12),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_12r=plot!(Delta_x.*(-1),vec(r_right12),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_12v=Plots.plot();
p_12v=plot!(Delta_x.*(-1),vec(v_left12),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_12v=plot!(Delta_x.*(-1),vec(v_right12),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
plot(p_12r,p_12v,size=(900,400))

In [ ]:
t_12=Table(Δx=Delta_x,ratio_left=vec(r_left12),mean_left=vec(v_left12),ratio_right=vec(r_right12),mean_right=vec(v_right12))

For $x=13$, we have

In [ ]:
Delta_x=[-1,-0.5,-0.2,-0.1,-0.05,-0.04,-0.02,-0.01,-0.005,-0.004,-0.002,-0.001,-0.0005,-0.0002,-0.0001];
r_left13=zeros(1,length(Delta_x));r_right13=zeros(1,length(Delta_x));
v_left13=zeros(1,length(Delta_x));v_right13=zeros(1,length(Delta_x));
α = [11/6,-3,3/2,-1/3];β = [1,0,0,0];
C=A+13*B;
Eig=eigvals(Matrix(C));
for j=1:length(Delta_x)
    z_left=zeros(ComplexF64,0);z_right=zeros(ComplexF64,0);
    ξ_left=zeros(ComplexF64,0);ξ_right=zeros(ComplexF64,0);
    Δx=Delta_x[j]
    z=Δx*Eig;
    for i=1:length(z)
        ξ=compute_roots(α,β,z[i])
        if real(z[i])>0 && check_condition(ξ)==0
            append!(z_right,z[i])
            append!(ξ_right,ξ)
        end
        if real(z[i])<0 && check_condition(ξ)==0
            append!(z_left,z[i])
            append!(ξ_left,ξ)
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.004
            append!(Z_left,z[i])
        end
        if real(z[i])>0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_right,z[i])
        end
        if real(z[i])<0 && check_condition(ξ)==0 && Δx==-0.002
            append!(Z_left,z[i])
        end
    end
    r_left13[j]=length(z_left)/M_s;
    r_right13[j]=length(z_right)/M_s;
    if length(ξ_left)==0
        v_left13[j]=0
    end
    if length(ξ_right)==0
        v_right13[j]=0
    end
    if length(ξ_left)>0
        ξ_restl=check_condition2(ξ_left)
        μ_left=abs.((abs.(ξ_restl).-1)./Δx)
        v_left13[j]=maximum(μ_left)
    end
    if length(ξ_right)>0
        ξ_restr=check_condition2(ξ_right)
        μ_right=abs.((abs.(ξ_restr).-1)./Δx)
        v_right13[j]=maximum(μ_right)
    end
end

In [ ]:
p_13r=Plots.plot();
p_13r=plot!(Delta_x.*(-1),vec(r_left13),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_13r=plot!(Delta_x.*(-1),vec(r_right13),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_13v=Plots.plot();
p_13v=plot!(Delta_x.*(-1),vec(v_left13),lw=4,label="Left-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
p_13v=plot!(Delta_x.*(-1),vec(v_right13),lw=4,label="Right-half plane",legend=:topright,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log);
plot(p_13r,p_13v,size=(900,400))

In [ ]:
t_13=Table(Δx=Delta_x,ratio_left=vec(r_left13),mean_left=vec(v_left13),ratio_right=vec(r_right13),mean_right=vec(v_right13))

To determine the overall influence of the problematic eigenvalues, we combine the above results.

In [ ]:
p_c=Plots.plot();
r_left=((r_leftm10+r_leftm9+r_leftm8+r_leftm7+r_leftm6+r_leftm5+r_leftm4+r_leftm3+r_leftm2+r_leftm1+r_left0+r_left1+r_left2
        +r_left3+r_left4+r_left5+r_left6+r_left7+r_left8+r_left9+r_left10+r_left11+r_left12+r_left13)*M_s) ./ (M_s*24);
r_right=((r_rightm10+r_rightm9+r_rightm8+r_rightm7+r_rightm6+r_rightm5+r_rightm4+r_rightm3+r_rightm2+r_rightm1+r_right0+r_right1
        +r_right2+r_right3+r_right4+r_right5+r_right6+r_right7+r_right8+r_right9+r_right10+r_right11
        +r_right12+r_right13)*M_s) ./ (M_s*24);
v_left=(v_leftm10+v_leftm9+v_leftm8+v_leftm7+v_leftm6+v_leftm5+v_leftm4+v_leftm3+v_leftm2+v_leftm1+v_left0+v_left1+v_left2
        +v_left3+v_left4+v_left5+v_left6+v_left7+v_left8+v_left9+v_left10+v_left11+v_left12+v_left13) ./ 24;
v_right=(v_rightm10+v_rightm9+v_rightm8+v_rightm7+v_rightm6+v_rightm5+v_rightm4+v_rightm3+v_rightm2+v_rightm1+v_right0+v_right1
        +v_right2+v_right3+v_right4+v_right5+v_right6+v_right7+v_right8+v_right9+v_right10+v_right11+v_right12+v_right13) ./ 24;
for i=1:length(r_left)
    if r_left[i]==0
        r_left[i]=10^(-16)
    end
    if r_right[i]==0
        r_right[i]=10^(-16)
    end
    if v_left[i]==0
        v_left[i]=10^(-16)
    end
    if v_right[i]==0
        v_right[i]=10^(-16)
    end
end

In [ ]:
p_r=Plots.plot();
p_r=plot!(Delta_x.*(-1),vec(r_left),lw=4,label="Left-half plane",legend=:bottomleft,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log,yaxis=:log);
p_r=plot!(Delta_x.*(-1),vec(r_right),lw=4,label="Right-half plane",legend=:bottomleft,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log,yaxis=:log)

In [ ]:
p_v=Plots.plot();
p_v=plot!(Delta_x.*(-1),vec(v_left),lw=4,label="Left-half plane",legend=:bottomleft,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log,yaxis=:log);
p_v=plot!(Delta_x.*(-1),vec(v_right),lw=4,label="Right-half plane",legend=:bottomleft,marker=(:circle,5),xtickfontsize=15,
    ytickfontsize=15,legendfontsize=10,xaxis=:log,yaxis=:log)

In [ ]:
β=2;
TW2=TracyWidom;
xx=-2;
Delta_x=[-1,-0.5,-0.2,-0.1,-0.05,-0.04,-0.02,-0.01,-0.005,-0.004,-0.002,-0.001,-0.0005,-0.0002,-0.0001];
err=zeros(1,length(Delta_x));
for i=1:length(Delta_x)
    dx=Delta_x[i];
    result_cdf=TW(β;method="spectral",step="bdf3",interp=false,Δx_s=dx);
    x_cdf=result_cdf[1];F_cdf=result_cdf[2];
    ind3=findall(x -> x == xx, x_cdf)[1];
    err[i]=abs(F_cdf[ind3]-cdf(TW2,xx;beta=2,num_points=300));
end
for j=1:length(err)
    if err[j]>1
        err[j]=1
    end
end

In [ ]:
p_e=Plots.plot();
p_e=plot!(Delta_x.*(-1),vec(err),lw=4,legend=false,marker=(:circle,5),xtickfontsize=15,ytickfontsize=15,legendfontsize=10,
    xaxis=:log,yaxis=:log)

In [ ]:
function convergence_stability(α,β)
    θ = 0:0.01(1+rand()/10):2*π # random perturbation to avoid singularities
    z = map(t -> R(α,β,exp(1im*t)),θ);
    xrange=[-1.,7.];
    yrange=[-4.,4.];
    contourf(xrange[1]:0.01:xrange[2],yrange[1]:0.01(1+rand()/10):yrange[2],(x,y)-> root_condition(α,β,x+1im*y),colorbar=false,
        xtickfontsize=15,ytickfontsize=15,legendfontsize=15)
    plot!(real(z),imag(z),xlim=xrange,ylim=yrange,aspectratio=1,legend=false,lw=5,linecolor=:orange,xtickfontsize=15,ytickfontsize=15,
        legendfontsize=10)
end
α = [11/6,-3,3/2,-1/3];β = [1,0,0,0];
plot_small=convergence_stability(α,β);
plot_small=Plots.scatter!(Z_left,c=:white,lw=0.005,xlabel="",ylabel="",xtickfontsize=15,ytickfontsize=15,legendfontsize=10);
plot_small=Plots.scatter!(Z_right,c=:red,lw=0.005,xlabel="",ylabel="",xtickfontsize=15,ytickfontsize=15,legendfontsize=10)

In [ ]:
function convergence_stability(α,β)
    θ = 0:0.01(1+rand()/10):2*π # random perturbation to avoid singularities
    z = map(t -> R(α,β,exp(1im*t)),θ);
    xrange=[-2.,2.];
    yrange=[-2.,2.];
    contourf(xrange[1]:0.01:xrange[2],yrange[1]:0.01(1+rand()/10):yrange[2],(x,y)-> root_condition(α,β,x+1im*y),colorbar=false,
        xtickfontsize=15,ytickfontsize=15,legendfontsize=15)
    plot!(real(z),imag(z),xlim=xrange,ylim=yrange,aspectratio=1,legend=false,lw=5,linecolor=:orange,xtickfontsize=15,ytickfontsize=15,
        legendfontsize=10)
end
α = [11/6,-3,3/2,-1/3];β = [1,0,0,0];
plot_large=convergence_stability(α,β);
plot_large=Plots.scatter!(Z_left,c=:white,lw=0.0001,xlabel="",ylabel="",xtickfontsize=15,ytickfontsize=15,legendfontsize=10);
plot_large=Plots.scatter!(Z_right,c=:red,lw=0.0001,xlabel="",ylabel="",xtickfontsize=15,ytickfontsize=15,legendfontsize=10)

The gap is about $[-0.004,-0.002]$ for BDF3.